## ABSA with ATE —> DeBERTa pipeline

In [ ]:
%pip install spacy transformers seaborn matplotlib pandas

In [4]:
import spacy
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Spacy model
nlp = spacy.load("en_core_web_sm")

# Pre-trained DeBERTa model for absa
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)


/Users/yusuf/dev/models/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/yusuf/dev/models/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/yusuf/dev/models/.venv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens in

In [5]:
def extract_aspects(text):
    doc = nlp(text)
    aspects = []
    for chunk in doc.noun_chunks:  # extract noun phrases
        if any(token.dep_ in ("nsubj", "dobj") for token in chunk):  # focus on key aspects
            aspects.append(chunk.text)  # store the full noun phrase
    return aspects

In [6]:
def analyze_aspect_sentiment(text, aspects):
    sentiment_results = {}
    for aspect in aspects:
        result = classifier(text, text_pair=aspect)[0]  # get full result dict
        sentiment = result['label']
        confidence = round(result['score'], 5)  # round to 5 digits
        sentiment_results[aspect] = {'sentiment': sentiment, 'confidence': confidence}
    return sentiment_results

In [7]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

def evaluate_model_on_csv(df):
    y_true = []
    y_pred = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        sentence = row['Sentence']
        aspect = row['Aspect Term']
        true_label = row['polarity'].lower()

        input_text = f"{aspect} in '{sentence}' is"
        pred_label = classifier(input_text)[0]['label'].lower()

        y_true.append(true_label)
        y_pred.append(pred_label)

    # Report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, digits=4))

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred, labels=["positive", "negative", "neutral"])
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=["positive", "negative", "neutral"],
                yticklabels=["positive", "negative", "neutral"])
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()


In [ ]:
df = pd.read_csv("data/Laptop_Train_v2.csv")
evaluate_model_on_csv(df)

In [8]:
# example text
text = "The food was great, but the service was terrible."

# extract aspects
aspects = extract_aspects(text)

# get sentiment for each aspect
aspect_sentiments = analyze_aspect_sentiment(text, aspects)

In [9]:
aspect_sentiments

{'The food': {'sentiment': 'Positive', 'confidence': 0.99696},
 'the service': {'sentiment': 'Negative', 'confidence': 0.99194}}